In [33]:
# -*- coding: utf-8 -*-
import pandas as pd
import pymssql 
import datetime
from openpyxl import load_workbook
from openpyxl.styles import Font

# 数据处理
def decode_columns(result, column_names):
    """
    对pandas数据框中的多个列进行编解码，并返回新的数据框
    
    Args:
        chuda: pandas数据框
        column_names: list，需要进行编解码的列名列表
    
    Returns:
        pandas数据框，编解码后的结果
    """
    for column_name in column_names:
        # 跳过空值
        mask = result[column_name].notnull()
        # 对指定列进行编解码
        result.loc[mask, column_name] = result.loc[mask, column_name].apply(lambda x: x.encode('latin1').decode('gbk'))
    # 返回编解码后的结果
    return result
# 按照周份
def get_dynamic_dates_weekly():
    # 获取今天的日期
    today = datetime.date.today()
    # 获取今天是周几（0代表周一，1代表周二，依次类推）
    weekday = today.weekday()
    if weekday == 0: # 今天是周一
        # 开始时间设置为上周周一
        start_date = today - datetime.timedelta(days=7)
        # 结束时间设置为上周周末
        end_date = today - datetime.timedelta(days=1)
    else: # 今天不是周一
        # 开始时间设置为这周周一
        start_date = today - datetime.timedelta(days=16)
        # 结束时间设置为昨天
        end_date = today - datetime.timedelta(days=1)
    # 返回开始时间和结束时间
    return {start_date}, end_date
def get_dynamic_dates():
    # 获取今天的日期
    today = datetime.date.today()
    # 获取本月的第一天
    first_day_of_month = today.replace(day=1)
    # 判断今天是否在本月的3天内
    if today.day <4:
        # 如果是，将开始时间设置为上个月的1号
        start_date = (first_day_of_month - datetime.timedelta(days=1)).replace(day=1)
    else:
        # 如果不是，将开始时间设置为本月的1号
        start_date = first_day_of_month
    # 将结束时间设置为昨天
    end_date = today - datetime.timedelta(days=1)
    # 返回开始时间和结束时间
    return start_date, end_date


def get_db_connection(config):
    return pymssql.connect(config['server'], config['user'], config['password'], config['database'])

def execute_query_and_process_data(conn, sql, decode_columns_list):
    with conn.cursor(as_dict=True) as cursor:
        cursor.execute(sql)
        data = pd.DataFrame(cursor.fetchall())
        data = decode_columns(data, decode_columns_list)
        for col in data.columns:
            if col not in decode_columns_list:
                data[col] = pd.to_numeric(data[col])
        return data

start_date, end_date = get_dynamic_dates()
params = {"start_date":start_date,"end_date":end_date}
round_func = lambda x: round(x, 2)
print(params["start_date"])
print(params["end_date"])

2023-11-01
2023-11-26


## 原始数据提取

In [75]:
db_config = {
    'server': '192.168.0.169',
    'user': 'chjreport',
    'password': 'Chj@12345',
    'database': 'ChjBidb'
}
sql = f'''
        SELECT 
            a.大区,
            a.小区,
            a.门店代码,
            a.门店名称,
            a.[门店类型],
            SUM(a.消费人数) AS 总消费人数,
            SUM(a.消费会员企微添加数) AS 总消费会员企微添加数,
            SUM(a.新会员消费人数) AS 总新会员消费人数,
            SUM(a.新会员企微添加数) AS 总新会员企微添加数,
            SUM(a.老会员消费人数) AS 总老会员消费人数,
            SUM(a.老会员企微添加数) AS 总老会员企微添加数
        FROM v_shop_dict s
        LEFT JOIN (
            SELECT 
                c.大区,
                c.小区,
                c.[门店类型],
                c.门店代码,
                c.门店名称,
                COUNT(DISTINCT c.会员号) AS 消费人数,
                COUNT(DISTINCT CASE WHEN m.注册时间 BETWEEN '{start_date}' AND '{end_date}'+' 23:59:59' THEN c.会员号 ELSE NULL END) AS 新会员消费人数,
                COUNT(DISTINCT CASE WHEN m.注册时间 < '{start_date}' THEN c.会员号 ELSE NULL END) AS 老会员消费人数,
                COUNT(DISTINCT q.unionId) AS 消费会员企微添加数,
                COUNT(DISTINCT CASE WHEN m.注册时间 BETWEEN '{start_date}' AND '{end_date}'+' 23:59:59' THEN q.unionId ELSE NULL END) AS 新会员企微添加数,
                COUNT(DISTINCT CASE WHEN m.注册时间 < '{start_date}' THEN q.unionId ELSE NULL END) AS 老会员企微添加数
            FROM bi_business_member AS m 
            JOIN bi_business_consume AS c ON m.会员号 = c.会员号 
            JOIN v_btgoods AS g ON g.商品代码 = c.商品代码 
            LEFT JOIN (SELECT DISTINCT 企微unionId AS unionId FROM BI_Business_qywechatmember_all WHERE 企微添加时间 <= '{end_date}'+' 23:59:59') q ON m.微信unionId = q.unionId
            WHERE c.来源 IN ('智慧云店','智能中台','中台') 
                AND c.消费时间 BETWEEN '{start_date}' AND '{end_date}'+' 23:59:59' 
                AND g.二级大类 NOT IN ('其他','促销物料') 
                AND c.金额 > 0 
                AND c.商品代码 NOT IN ('SHG30000905','Q3G30000105','QQG30000091','QQG30000090','SHG30000943','SHG30001192') 
                AND m.是否在云店上激活 = '是'
            GROUP BY c.大区, c.小区, c.[门店类型], c.门店代码, c.门店名称
        ) as a ON a.门店代码 = s.门店代码 
        LEFT JOIN (
            SELECT ltrim(rtrim(t.firstShopCode)) as 门店代码, count(distinct t.mobile) as 门店吸粉数
            FROM v_mall_member t
            WHERE t.date_add BETWEEN '{start_date}' AND '{end_date}'+' 23:59:59'
                AND t.firstguideno NOT IN ('84e67d1115c859c336','2f7e946583048a96fe')  
            GROUP BY ltrim(rtrim(t.firstShopCode))
        ) as b ON s.门店代码 = b.门店代码
        WHERE (a.门店代码 IS NOT NULL OR b.门店代码 IS NOT NULL) and 大区 is not null and 大区 != '营销管理部'
        GROUP BY a.大区, a.小区, a.门店名称, a.[门店类型], a.门店代码
'''
task_details = [
    {"task_name": "消费7天", "decode_columns_list": ["大区","小区","门店名称","门店类型","门店代码"]}
]
with get_db_connection(db_config) as conn:
    for task in task_details:
        result = execute_query_and_process_data(conn, sql, task["decode_columns_list"])

## 门店明细

In [80]:
import pandas as pd

# 定义计算比率的函数
def calculate_rates(group):
    group['消费会员企微添加率'] = group['总消费会员企微添加数'] / group['总消费人数'] if group['总消费人数'].sum() > 0 else 0
    group['消费新客企微添加率'] = group['总新会员企微添加数'] / group['总新会员消费人数'] if group['总新会员消费人数'].sum() > 0 else 0
    group['消费老客企微添加率'] = group['总老会员企微添加数'] / group['总老会员消费人数'] if group['总老会员消费人数'].sum() > 0 else 0
    return group

# 对原始数据按照门店类型分组，并计算每组的比率
grouped_result = result.groupby('门店类型').apply(calculate_rates)
grouped_result
order = ['大区','小区','门店代码','门店名称','门店类型', '总消费人数','总消费会员企微添加数', '消费会员企微添加率', '总新会员消费人数', '总新会员企微添加数', '消费新客企微添加率', '总老会员消费人数','总老会员企微添加数', '消费老客企微添加率']
# # 请根据您实际的列名进行调整
sorted_result = grouped_result[order]
sorted_result


C:\Users\xieyu\AppData\Local\Temp\ipykernel_31544\1718098125.py:11: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  grouped_result = result.groupby('门店类型').apply(calculate_rates)


,大区,小区,门店代码,门店名称,门店类型,总消费人数,总消费会员企微添加数,消费会员企微添加率,总新会员消费人数,总新会员企微添加数,消费新客企微添加率,总老会员消费人数,总老会员企微添加数,消费老客企微添加率
0,北方分公司,黑龙江区-黑龙江A区,8A12,哈尔滨哈西万达,代理,156,30,0.192308,111,8,0.072072,45,22,0.488889
1,北方分公司,黑龙江区-黑龙江A区,8A02,哈尔滨红博会展购物,自营,57,40,0.701754,20,19,0.950000,37,21,0.567568
2,北方分公司,黑龙江区-黑龙江A区,8A19,哈尔滨红博西城红场,代理,144,118,0.819444,81,73,0.901235,63,45,0.714286
3,北方分公司,黑龙江区-黑龙江A区,8A16,哈尔滨乐松,代理,93,86,0.924731,55,53,0.963636,38,33,0.868421
4,北方分公司,黑龙江区-黑龙江A区,8A17,哈尔滨南岗苏宁易购,代理,91,87,0.956044,53,53,1.000000,38,34,0.894737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,粤东区,粤东B区,KD03,汕头碧霞庄,自营,67,50,0.746269,15,13,0.866667,52,37,0.711538
1311,粤东区,粤东B区,KD13,汕头尚海阳光,代理,11,2,0.181818,4,0,0.000000,7,2,0.285714
1312,粤东区,粤东B区,KD04,汕头世贸花园,代理,40,21,0.525000,11,9,0.818182,29,12,0.413793
1313,粤东区,粤东B区,KD14,汕头小公园旗舰店,自营,122,24,0.196721,61,5,0.081967,61,19,0.311475


## 大区添加率

In [82]:
# 计算比率的函数
def calculate_rates(row):
    row['消费会员企微添加率'] = row['总消费会员企微添加数'] / row['总消费人数'] if row['总消费人数'] > 0 else 0
    row['消费新客企微添加率'] = row['总新会员企微添加数'] / row['总新会员消费人数'] if row['总新会员消费人数'] > 0 else 0
    row['消费老客企微添加率'] = row['总老会员企微添加数'] / row['总老会员消费人数'] if row['总老会员消费人数'] > 0 else 0
    return row
# 应用计算比率的函数
result_daqu = result.apply(calculate_rates, axis=1)

# 按大区和门店类型分组，计算平均值
grouped = result_daqu.groupby(['大区', '门店类型']).agg({'消费会员企微添加率': 'mean', '消费新客企微添加率': 'mean', '消费老客企微添加率': 'mean'}).unstack(fill_value=0)

# 计算每个大区的整体比率
def calculate_total_rates(group):
    total_consumers = group['总消费人数'].sum()
    total_new_members = group['总新会员消费人数'].sum()
    total_old_members = group['总老会员消费人数'].sum()

    total_added_members = group['总消费会员企微添加数'].sum()
    total_added_new_members = group['总新会员企微添加数'].sum()
    total_added_old_members = group['总老会员企微添加数'].sum()

    group['整体_消费会员企微添加率'] = total_added_members / total_consumers if total_consumers > 0 else 0
    group['整体_消费新客企微添加率'] = total_added_new_members / total_new_members if total_new_members > 0 else 0
    group['整体_消费老客企微添加率'] = total_added_old_members / total_old_members if total_old_members > 0 else 0
    return group
# 应用整体比率计算
total_rates = result_daqu.groupby('大区').apply(calculate_total_rates).groupby('大区').agg({'整体_消费会员企微添加率': 'mean', '整体_消费新客企微添加率': 'mean', '整体_消费老客企微添加率': 'mean'})

# 合并分组结果和整体比率
grouped = pd.concat([grouped, total_rates], axis=1)


# 首先，将"整体"指标列转换为多级索引格式
grouped[('整体', '消费会员企微添加率')] = grouped['整体_消费会员企微添加率']
grouped[('整体', '消费新客企微添加率')] = grouped['整体_消费新客企微添加率']
grouped[('整体', '消费老客企微添加率')] = grouped['整体_消费老客企微添加率']

# 然后，使用reindex重新排列列
new_columns = [
    ('整体', '消费会员企微添加率'),
    ('整体', '消费新客企微添加率'),
    ('整体', '消费老客企微添加率'),
    ('消费会员企微添加率', '自营'),
    ('消费新客企微添加率', '自营'),
    ('消费老客企微添加率', '自营'),
    ('消费会员企微添加率', '代理'),
    ('消费新客企微添加率', '代理'),
    ('消费老客企微添加率', '代理')
]

grouped = grouped.reindex(columns=new_columns)
grouped.reset_index(inplace=True)
grouped = grouped.sort_values(by=('整体', '消费会员企微添加率'), ascending=True)
# 对自营门店数据进行筛选
result_self_operated = result[result['门店类型'] == '自营']

# 对代理门店数据进行筛选
result_agent = result[result['门店类型'] == '代理']

# 计算自营门店的比率
total_data_self_operated = result_self_operated.agg({
    '总消费人数': 'sum',
    '总消费会员企微添加数': 'sum',
    '总新会员消费人数': 'sum',
    '总新会员企微添加数': 'sum',
    '总老会员消费人数': 'sum',
    '总老会员企微添加数': 'sum'
})

# 计算代理门店的比率
total_data_agent = result_agent.agg({
    '总消费人数': 'sum',
    '总消费会员企微添加数': 'sum',
    '总新会员消费人数': 'sum',
    '总新会员企微添加数': 'sum',
    '总老会员消费人数': 'sum',
    '总老会员企微添加数': 'sum'
})
total_data = result.agg({
    '总消费人数': 'sum',
    '总消费会员企微添加数': 'sum',
    '总新会员消费人数': 'sum',
    '总新会员企微添加数': 'sum',
    '总老会员消费人数': 'sum',
    '总老会员企微添加数': 'sum'
})
# 计算全国水平的比率
total_rates = {
    ('整体', '消费会员企微添加率'): total_data['总消费会员企微添加数'] / total_data['总消费人数'] if total_data['总消费人数'] > 0 else 0,
    ('整体', '消费新客企微添加率'): total_data['总新会员企微添加数'] / total_data['总新会员消费人数'] if total_data['总新会员消费人数'] > 0 else 0,
    ('整体', '消费老客企微添加率'): total_data['总老会员企微添加数'] / total_data['总老会员消费人数'] if total_data['总老会员消费人数'] > 0 else 0,
    ('消费会员企微添加率', '自营'): total_data_self_operated['总消费会员企微添加数'] / total_data_self_operated['总消费人数'] if total_data_self_operated['总消费人数'] > 0 else 0,
    ('消费新客企微添加率', '自营'): total_data_self_operated['总新会员企微添加数'] / total_data_self_operated['总新会员消费人数'] if total_data_self_operated['总新会员消费人数'] > 0 else 0,
    ('消费老客企微添加率', '自营'): total_data_self_operated['总老会员企微添加数'] / total_data_self_operated['总老会员消费人数'] if total_data_self_operated['总老会员消费人数'] > 0 else 0,
    ('消费会员企微添加率', '代理'): total_data_agent['总消费会员企微添加数'] / total_data_agent['总消费人数'] if total_data_agent['总消费人数'] > 0 else 0,
    ('消费新客企微添加率', '代理'): total_data_agent['总新会员企微添加数'] / total_data_agent['总新会员消费人数'] if total_data_agent['总新会员消费人数'] > 0 else 0,
    ('消费老客企微添加率', '代理'): total_data_agent['总老会员企微添加数'] / total_data_agent['总老会员消费人数'] if total_data_agent['总老会员消费人数'] > 0 else 0
}

# 创建表示全国水平的行
national_row = pd.DataFrame([total_rates], index=["全国"])

# 将全国水平的行添加到grouped DataFrame中
grouped = pd.concat([grouped, national_row])

# 重新展示DataFrame
grouped

C:\Users\xieyu\AppData\Local\Temp\ipykernel_31544\3882499586.py:28: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  total_rates = result_daqu.groupby('大区').apply(calculate_total_rates).groupby('大区').agg({'整体_消费会员企微添加率': 'mean', '整体_消费新客企微添加率': 'mean', '整体_消费老客企微添加率': 'mean'})


,大区,"(整体, 消费会员企微添加率)","(整体, 消费新客企微添加率)","(整体, 消费老客企微添加率)","(消费会员企微添加率, 自营)","(消费新客企微添加率, 自营)","(消费老客企微添加率, 自营)","(消费会员企微添加率, 代理)","(消费新客企微添加率, 代理)","(消费老客企微添加率, 代理)"
9,粤东区,0.459000,0.423773,0.481240,0.489837,0.539183,0.481078,0.353409,0.409091,0.349754
5,华南分公司,0.533386,0.589390,0.481982,0.681651,0.769239,0.621781,0.472054,0.462361,0.472524
2,华东分公司,0.573101,0.637156,0.526475,0.642107,0.781093,0.585362,0.511700,0.538896,0.470666
10,西北大区,0.576107,0.603987,0.549720,0.619992,0.719182,0.550462,0.573349,0.591715,0.559808
4,华北分公司,0.605562,0.718057,0.520420,0.611087,0.720016,0.517485,0.570201,0.627034,0.519324
6,江苏分公司,0.634418,0.776158,0.566649,0.662431,0.825443,0.596651,0.613666,0.704227,0.566283
3,华中大区,0.661241,0.731039,0.602026,0.757138,0.987179,0.651357,0.662740,0.720606,0.628390
0,内蒙大区,0.701802,0.786290,0.633550,0.860368,0.922064,0.805253,0.498974,0.481855,0.461826
8,海南大区,0.706835,0.723906,0.687259,0.000000,0.000000,0.000000,0.844626,0.876756,0.813073
11,西南分公司,0.727072,0.776672,0.668767,0.675960,0.741925,0.608210,0.684046,0.699913,0.655908


## 小区添加率

In [57]:
# 应用计算比率的函数
result_xiaoqu = result.apply(calculate_rates, axis=1)

# 按大区和门店类型分组，计算平均值
grouped2 = result_xiaoqu.groupby(['小区', '门店类型']).agg({'消费会员企微添加率': 'mean', '消费新客企微添加率': 'mean', '消费老客企微添加率': 'mean'}).unstack(fill_value=0)

# 计算每个大区的整体比率
def calculate_total_rates(group):
    total_consumers = group['总消费人数'].sum()
    total_new_members = group['总新会员消费人数'].sum()
    total_old_members = group['总老会员消费人数'].sum()

    total_added_members = group['总消费会员企微添加数'].sum()
    total_added_new_members = group['总新会员企微添加数'].sum()
    total_added_old_members = group['总老会员企微添加数'].sum()

    group['整体_消费会员企微添加率'] = total_added_members / total_consumers if total_consumers > 0 else 0
    group['整体_消费新客企微添加率'] = total_added_new_members / total_new_members if total_new_members > 0 else 0
    group['整体_消费老客企微添加率'] = total_added_old_members / total_old_members if total_old_members > 0 else 0
    return group
# 应用整体比率计算
total_rates = result_xiaoqu.groupby('小区').apply(calculate_total_rates).groupby('小区').agg({'整体_消费会员企微添加率': 'mean', '整体_消费新客企微添加率': 'mean', '整体_消费老客企微添加率': 'mean'})

# 合并分组结果和整体比率
grouped2 = pd.concat([grouped2, total_rates], axis=1)
# 首先，将"整体"指标列转换为多级索引格式
grouped2[('整体', '消费会员企微添加率')] = grouped2['整体_消费会员企微添加率']
grouped2[('整体', '消费新客企微添加率')] = grouped2['整体_消费新客企微添加率']
grouped2[('整体', '消费老客企微添加率')] = grouped2['整体_消费老客企微添加率']

# 然后，使用reindex重新排列列
new_columns = [
    ('整体', '消费会员企微添加率'),
    ('整体', '消费新客企微添加率'),
    ('整体', '消费老客企微添加率'),
    ('消费会员企微添加率', '自营'),
    ('消费新客企微添加率', '自营'),
    ('消费老客企微添加率', '自营'),
    ('消费会员企微添加率', '代理'),
    ('消费新客企微添加率', '代理'),
    ('消费老客企微添加率', '代理')
]

grouped2 = grouped2.reindex(columns=new_columns)
# 去除索引列
grouped2.reset_index(inplace=True)
grouped2
# 按照整体_消费会员企微添加率从小到大排序，取20个
grouped2 = grouped2.sort_values(by=('整体', '消费会员企微添加率'), ascending=True).head(20)
grouped2

C:\Users\xieyu\AppData\Local\Temp\ipykernel_31544\2100650206.py:22: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  total_rates = result_xiaoqu.groupby('小区').apply(calculate_total_rates).groupby('小区').agg({'整体_消费会员企微添加率': 'mean', '整体_消费新客企微添加率': 'mean', '整体_消费老客企微添加率': 'mean'})


,小区,"(整体, 消费会员企微添加率)","(整体, 消费新客企微添加率)","(整体, 消费老客企微添加率)","(消费会员企微添加率, 自营)","(消费新客企微添加率, 自营)","(消费老客企微添加率, 自营)","(消费会员企微添加率, 代理)","(消费新客企微添加率, 代理)","(消费老客企微添加率, 代理)"
30,安徽E区,0.139535,0.126506,0.144495,0.000000,0.000000,0.000000,0.187380,0.177111,0.109152
55,河南A区,0.212099,0.116803,0.266745,0.000000,0.000000,0.000000,0.255771,0.138742,0.344141
56,河南B区,0.227027,0.178914,0.262295,0.000000,0.000000,0.000000,0.223239,0.151603,0.259243
73,淮安区,0.295652,0.388753,0.244265,0.590303,0.866667,0.446474,0.198215,0.192643,0.202442
87,营销其他（本部）,0.333333,0.166667,0.363636,0.507937,0.041667,0.506313,0.000000,0.000000,0.000000
62,济南B区,0.356143,0.440191,0.315668,0.521155,0.497229,0.513833,0.308260,0.262976,0.315381
84,粤东B区,0.379310,0.306306,0.439114,0.407946,0.411006,0.429259,0.353409,0.409091,0.349754
1,上海B区,0.396605,0.387205,0.404558,0.466667,0.833333,0.375000,0.378938,0.344882,0.387221
106,青岛E区,0.397015,0.489510,0.328125,0.000000,0.000000,0.000000,0.417897,0.470384,0.386401
69,浙江-杭州D区,0.398784,0.419483,0.382716,0.514471,0.466984,0.557802,0.277298,0.292383,0.255958


In [58]:
import pandas as pd

# 假设 result 是您的原始数据集
# ... 您之前的代码 ...

# 应用计算比率的函数
result = result.apply(calculate_rates, axis=1)

# 按门店名称和门店类型分组，计算平均值
grouped4 = result.groupby(['门店名称', '门店类型']).agg({'消费会员企微添加率': 'mean', '消费新客企微添加率': 'mean', '消费老客企微添加率': 'mean'}).unstack(fill_value=0)

# 为每个门店计算整体比率
def calculate_total_rates(group):
    total_rates = {
        ('整体', '消费会员企微添加率'): group['消费会员企微添加率'].mean(),
        ('整体', '消费新客企微添加率'): group['消费新客企微添加率'].mean(),
        ('整体', '消费老客企微添加率'): group['消费老客企微添加率'].mean()
    }
    return pd.Series(total_rates)

total_rates = result.groupby('门店名称').apply(calculate_total_rates)

# 将整体比率添加到grouped4中，并保持列的多级索引结构
grouped4 = pd.concat([grouped4, total_rates], axis=1)

# 重新排列列，确保使用正确的多级索引格式
columns_order = [
    ('整体', '消费会员企微添加率'),
    ('整体', '消费新客企微添加率'),
    ('整体', '消费老客企微添加率'),
    ('消费会员企微添加率', '自营'),
    ('消费新客企微添加率', '自营'),
    ('消费老客企微添加率', '自营'),
    ('消费会员企微添加率', '代理'),
    ('消费新客企微添加率', '代理'),
    ('消费老客企微添加率', '代理')
]

grouped4 = grouped4[columns_order]

# 重置索引并展示
grouped4.reset_index(inplace=True)
grouped4


门店名称        整体                     消费会员企微添加率 消费新客企微添加率  \
门店类型                    消费会员企微添加率 消费新客企微添加率 消费老客企微添加率        自营        自营   
0     SouffleX上海盒马X会员森兰  1.000000  0.000000  1.000000  0.000000  0.000000   
1         Souffle上海西郊百联  0.130435  0.125000  0.133333  0.000000  0.000000   
2           Souffle南京中央  0.803571  0.854545  0.754386  0.803571  0.854545   
3          Souffle南宁万象城  0.215517  0.217391  0.214286  0.215517  0.217391   
4          Souffle南阳万悦城  0.914286  0.938776  0.857143  0.000000  0.000000   
...                 ...       ...       ...       ...       ...       ...   
1314           （已撤）丽水百大  0.666667  1.000000  0.000000  0.000000  0.000000   
1315         （已撤）南京大洋桥北  1.000000  0.000000  1.000000  1.000000  0.000000   
1316           （已撤）永康宝龙  0.882353  1.000000  0.818182  0.882353  1.000000   
1317     （已撤）鄂尔多斯薛家湾专营店  0.125000  0.000000  0.125000  0.000000  0.000000   
1318         （已撤）阜阳颍泉万达  0.250000  0.000000  0.250000  0.000000  0.000000   

     消费老客企微添加率 消费会员企微添加率 消费新客企微添加率 消费老客企微添加率  
门店类型        自营        代理        代理        代理  
0     0.000000  1.000000  0.000000  1.000000  
1     0.000000  0.130435  0.125000  0.133333  
2     0.754386  0.000000  0.000000  0.000000  
3     0.214286  0.000000  0.000000  0.000000  
4     0.000000  0.914286  0.938776  0.857143  
...        ...       ...       ...       ...  
1314  0.000000  0.666667  1.000000  0.000000  
1315  1.000000  0.000000  0.000000  0.000000  
1316  0.818182  0.000000  0.000000  0.000000  
1317  0.000000  0.125000  0.000000  0.125000  
1318  0.000000  0.250000  0.000000  0.250000  

[1319 rows x 10 columns]

## 导出文档

In [81]:
from openpyxl import load_workbook
from openpyxl.styles import Alignment
# 读取数据
# 假设你的 DataFrame 名称为 result2
# result2 = pd.DataFrame(...)
# 加载现有的 Excel 文件
file_path = r'D:\企微添加率监控测试1.0.xlsx'
book = load_workbook(file_path)

# 大区统计表
sheet_name1 = '大区情况'  # 请更改为你的工作表名称
sheet1 = book[sheet_name1]
# 将 DataFrame 数据写入工作表的指定位置
start_row = 4  # 起始行
start_col = 1  # 起始列
for index, row in enumerate(grouped.iterrows()):
    for j, value in enumerate(row[1]):  # 注意这里的变化：使用 row[1] 而不是 row
        sheet1.cell(row=start_row + index, column=start_col + j, value=value)
# 把从第三行起和第二列起的格式都设置为百分数
for i in range(3, sheet1.max_row + 1):
    for j in range(2, sheet1.max_column + 1):
        sheet1.cell(row=i, column=j).number_format = '0.0%'
        sheet1.cell(row=i, column=j).alignment = Alignment(horizontal='center', vertical='center')

# 落后小区
sheet_name1 = '落后小区'  # 请更改为你的工作表名称
sheet1 = book[sheet_name1]
# 将 DataFrame 数据写入工作表的指定位置
start_row = 4  # 起始行
start_col = 1  # 起始列
for index, row in enumerate(grouped2.iterrows()):
    for j, value in enumerate(row[1]):  # 注意这里的变化：使用 row[1] 而不是 row
        sheet1.cell(row=start_row + index, column=start_col + j, value=value)
# 把从第三行起和第二列起的格式都设置为百分数
for i in range(4, sheet1.max_row + 1):
    for j in range(2, sheet1.max_column + 1):
        sheet1.cell(row=i, column=j).number_format = '0.0%'
        sheet1.cell(row=i, column=j).alignment = Alignment(horizontal='center', vertical='center')


# 数据明细表 
sheet_name3 = '门店情况'  # 请更改为你的工作表名称
sheet3 = book[sheet_name3]
# 将 DataFrame 数据写入工作表的指定位置
start_row = 4  # 起始行
start_col = 1  # 起始列
for index, row in enumerate(grouped4.iterrows()):
    for j, value in enumerate(row[1]):  # 注意这里的变化：使用 row[1] 而不是 row
        sheet3.cell(row=start_row + index, column=start_col + j, value=value)
# 把从第三行起和第二列起的格式都设置为百分数
for i in range(2, sheet3.max_row + 1):
    for j in range(12, sheet3.max_column + 1):
        sheet3.cell(row=i, column=j).number_format = '0.0%'
        sheet3.cell(row=i, column=j).alignment = Alignment(horizontal='center', vertical='center')
book.save(file_path)

# 数据明细表 
sheet_name3 = '数据明细'  # 请更改为你的工作表名称
sheet3 = book[sheet_name3]
# 将 DataFrame 数据写入工作表的指定位置
start_row = 2  # 起始行
start_col = 1  # 起始列
for index, row in enumerate(sorted_result.iterrows()):
    for j, value in enumerate(row[1]):  # 注意这里的变化：使用 row[1] 而不是 row
        sheet3.cell(row=start_row + index, column=start_col + j, value=value)
# 把从第三行起和第二列起的格式都设置为百分数
for i in range(2, sheet3.max_row + 1):
    for j in range(12, sheet3.max_column + 1):
        sheet3.cell(row=i, column=j).number_format = '0.0%'
        sheet3.cell(row=i, column=j).alignment = Alignment(horizontal='center', vertical='center')
book.save(file_path)
print('写入成功')

写入成功
